In [ ]:
import numpy as np
import pandas as pd
import tensorflow
import keras
from keras.models import Model
from keras.layers import Input, LSTM, Dense,TimeDistributed,Embedding,Bidirectional
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from string import digits
import nltk
import re
import string

# pd.set_option('display.max_rows', 500)
# pd.set_option('display.max_columns', 500)
# pd.set_option('display.width', 1000)
# pd.set_option('display.max_colwidth', -1)

In [2]:
import pandas as pd
import os

directory_path = r"C:\Users\Najeeb\Desktop\SIT Semesters\Sem 5\LAWWWWWWWWWWWWWWWWW\Dataset\nv3\nv3"

# Function to extract text between two patterns
def extract_text_between_patterns(text, start_pattern, end_pattern):
    start_index = text.find(start_pattern)
    end_index = text.find(end_pattern)
    if start_index != -1 and end_index != -1:
        return text[start_index + len(start_pattern):end_index]
    else:
        return None

# Create an empty DataFrame
data = {
    'Case Name': [],
    'Committee Members': [],
    'PROCEDURE': [],
    'THE FACTS': [],
    'THE LAW': [],
    'Judgment': []
}

df = pd.DataFrame(data)

# Loop through text files
for i in range(1, 561):  # Assuming you have files named 1.txt to 560.txt
    filename = f"{i}.txt"
    file_path = os.path.join(directory_path, filename)

    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()

    case_name = extract_text_between_patterns(text, "In the case of ", ",")
    committee_members = extract_text_between_patterns(text, ":", "Having deliberated")
    procedure_info = extract_text_between_patterns(text, "PROCEDURE", "THE FACTS")
    facts_info = extract_text_between_patterns(text, "THE FACTS", "THE LAW")
    law_info = extract_text_between_patterns(text, "THE LAW", "FOR THESE REASONS")
    judgment_info = extract_text_between_patterns(text, "FOR THESE REASONS", "Done in")

    # Append data to the DataFrame
    df = df.append({
        'Case Name': case_name,
        'Committee Members': committee_members,
        'PROCEDURE': procedure_info,
        'THE FACTS': facts_info,
        'THE LAW': law_info,
        'Judgment': judgment_info
    }, ignore_index=True)



C:\Users\Najeeb\AppData\Local\Temp\ipykernel_1520\628256216.py:43: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({
C:\Users\Najeeb\AppData\Local\Temp\ipykernel_1520\628256216.py:43: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({
C:\Users\Najeeb\AppData\Local\Temp\ipykernel_1520\628256216.py:43: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({
C:\Users\Najeeb\AppData\Local\Temp\ipykernel_1520\628256216.py:43: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({
C:\Users\Najeeb\AppData\Local\Temp\ipykernel_1520\628256216.py:43: FutureWarning: The frame.append method is deprecated and 

In [3]:
# df = df[:1]
# pd.set_option('display.max_colwidth', -1)
# df.head()

In [4]:
df.shape

(560, 6)

In [5]:
df["Committee Members"].isnull().sum()

11

In [6]:
df["Judgment"].isnull().sum()

15

In [7]:
df["THE LAW"].isnull().sum()


17

In [8]:
df["THE FACTS"].isnull().sum()


17

In [9]:
df["PROCEDURE"].isnull().sum()


14

In [10]:
df = df.dropna()

In [11]:
import re
import pandas as pd

def clean_text(text, patterns_to_remove):
    # Create a regular expression pattern by joining the provided patterns with '|'
    pattern = '|'.join(re.escape(pat) for pat in patterns_to_remove)
    
    # Remove the specified patterns except single newlines '\n'
    cleaned_text = re.sub(pattern, '', text)
    cleaned_text = re.sub(r'(\d+\.\s*)', ' ', cleaned_text)  # Remove digits and dots
    cleaned_text = re.sub(r'(\n{2,})', ' ', cleaned_text)  # Replace multiple newlines with a single space
    return cleaned_text.strip()  # Strip leading/trailing spaces


# Apply the clean_text function to the 'procedure' column
df['Case Name'] = df['Case Name'].apply(lambda x: clean_text(x, ['\n\n', '\n']))
df['Committee Members'] = df['Committee Members'].apply(lambda x: clean_text(x, ['\n\n', '\n']))
df['PROCEDURE'] = df['PROCEDURE'].apply(lambda x: clean_text(x, ['\n\n', '\n']))
df['THE FACTS'] = df['THE FACTS'].apply(lambda x: clean_text(x, ['\n\n', '\n']))
df['THE LAW'] = df['THE LAW'].apply(lambda x: clean_text(x, ['\n\n', '\n']))
df['Judgment'] = df['Judgment'].apply(lambda x: clean_text(x, ['\n\n', '\n']))

# Print the cleaned DataFrame
# df.head()


In [12]:
df.head()

,Case Name,Committee Members,PROCEDURE,THE FACTS,THE LAW,Judgment
0,Terge v. Hungary,"Faris Vehabović, President,Carlo Ranzoni,Péter...",The case originated in an application (no. 362...,I. THE CIRCUMSTANCES OF THE CASE The applican...,I. ALLEGED VIOLATION OF ARTICLE 3 OF THE CONV...,", THE COURT, UNANIMOUSLY, Declares the applica..."
1,Hristoskov v. Bulgaria,"Gabriele Kucsko-Stadlmayer, President,Yonko Gr...",The case originated in an application (no. 507...,I. THE CIRCUMSTANCES OF THE CASE The applican...,I. SCOPE OF THE CASE After notice of the appl...,", THE COURT, UNANIMOUSLY, Declares the complai..."
2,Pihoni v. Albania,"Robert Spano, President,Paul Lemmens,Ledi Bian...",The case originated in an application (no. 743...,I. THE CIRCUMSTANCES OF THE CASE The applican...,I. ALLEGED VIOLATION OF ARTICLES 3 AND 13 OF ...,", THE COURT UNANIMOUSLY Decides to join to the..."
3,Mskhiladze v. Russia,"Helen Keller, President,Pere Pastor Vilanova,A...",The case originated in an application (no. 477...,I. THE CIRCUMSTANCES OF THE CASE The applican...,I. ALLEGED VIOLATION OF ARTICLE 3 OF THE CONV...,", THE COURT, UNANIMOUSLY, Declares the complai..."
4,Akimenkov v. Russia,"Helena Jäderblom, President,Luis López Guerra,...",The case originated in two applications (nos. ...,I. THE CIRCUMSTANCES OF THE CASE The applican...,I. JOINDER OF THE APPLICATIONS Given their co...,", THE COURT, UNANIMOUSLY, Decides to join the ..."


In [ ]:
from nltk.corpus import stopwords
stop = stopwords.words('english')

def remove_starting_comma(text):
    return re.sub(r'^, ', '', text)

In [14]:
columns_to_clean = ['Case Name', 'Committee Members', 'PROCEDURE', 'THE FACTS', 'THE LAW', 'Judgment']

df[columns_to_clean] = df[columns_to_clean].applymap(lambda x: clean_text(x, ['\n\n', '\n']))
df[columns_to_clean] = df[columns_to_clean].applymap(lambda x: x.lower())
df[columns_to_clean] = df[columns_to_clean].applymap(lambda x: re.sub("'", '', x))
df['Judgment'] = df['Judgment'].apply(remove_starting_comma)
df[columns_to_clean] = df[columns_to_clean].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
df[columns_to_clean] = df[columns_to_clean].apply(lambda x: re.sub("'", '', x))

<font family="Arial, sans-serif" size="8px" color="#D2E0FB">CHAT GPT TEXT GENERATION</font>


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Attention
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseCategoricalAccuracy
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Assuming you have a DataFrame df with 'input_text' and 'Judgment' columns
input_texts = df['input_text'].astype(str).tolist()
judgments = df['Judgment'].astype(str).tolist()

# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(input_texts + judgments)
input_sequences = tokenizer.texts_to_sequences(input_texts)
judgment_sequences = tokenizer.texts_to_sequences(judgments)

# Pad sequences to a fixed length
max_sequence_length = 1000  # Adjust as needed
input_sequences = pad_sequences(input_sequences, maxlen=max_sequence_length, padding='post')
judgment_sequences = pad_sequences(judgment_sequences, maxlen=max_sequence_length, padding='post')

# Define the model architecture
embedding_dim = 256
hidden_units = 512

input_layer = Input(shape=(max_sequence_length,))
embedding_layer = Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=embedding_dim)(input_layer)
lstm_layer = LSTM(hidden_units, return_sequences=True)(embedding_layer)
attention_layer = Attention()([lstm_layer, lstm_layer])
output_layer = Dense(len(tokenizer.word_index) + 1, activation='softmax')(attention_layer)

model = Model(inputs=input_layer, outputs=output_layer)

optimizer = Adam(learning_rate=0.001)
loss_fn = SparseCategoricalCrossentropy()
accuracy_metric = SparseCategoricalAccuracy()

model.compile(optimizer=optimizer, loss=loss_fn, metrics=[accuracy_metric])

# Split data into training and validation sets
train_ratio = 0.8
split_index = int(len(input_sequences) * train_ratio)

input_train, input_val = input_sequences[:split_index], input_sequences[split_index:]
judgment_train, judgment_val = judgment_sequences[:split_index], judgment_sequences[split_index:]

# Define callbacks for training
early_stopping = EarlyStopping(patience=3, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('legal_text_generation_model.h5', save_best_only=True)

# Train the model
batch_size = 64
epochs = 20

# Now, you can use the trained model for inference on new cases, as described in the previous responses.


In [ ]:
model.fit(input_train, judgment_train, batch_size=batch_size, epochs=epochs,
        validation_data=(input_val, judgment_val),
        callbacks=[early_stopping, model_checkpoint])

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseCategoricalAccuracy
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Assuming you have a DataFrame df with 'input_text' and 'Judgment' columns
input_texts = df['input_text'].astype(str).tolist()
judgments = df['Judgment'].astype(str).tolist()

# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(input_texts + judgments)
input_sequences = tokenizer.texts_to_sequences(input_texts)
judgment_sequences = tokenizer.texts_to_sequences(judgments)

# Pad sequences to a fixed length
max_sequence_length = 1000  # Adjust as needed
input_sequences = pad_sequences(input_sequences, maxlen=max_sequence_length, padding='post')
judgment_sequences = pad_sequences(judgment_sequences, maxlen=max_sequence_length, padding='post')

# Define the model architecture
embedding_dim = 256
hidden_units = 512

input_layer = Input(shape=(max_sequence_length,))
embedding_layer = Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=embedding_dim)(input_layer)
lstm_layer = LSTM(hidden_units, return_sequences=True)(embedding_layer)
output_layer = Dense(len(tokenizer.word_index) + 1, activation='softmax')(lstm_layer)

model = Model(inputs=input_layer, outputs=output_layer)

optimizer = Adam(learning_rate=0.001)
loss_fn = SparseCategoricalCrossentropy()
accuracy_metric = SparseCategoricalAccuracy()

model.compile(optimizer=optimizer, loss=loss_fn, metrics=[accuracy_metric])

# Split data into training and validation sets
train_ratio = 0.8
split_index = int(len(input_sequences) * train_ratio)

input_train, input_val = input_sequences[:split_index], input_sequences[split_index:]
judgment_train, judgment_val = judgment_sequences[:split_index], judgment_sequences[split_index:]

# Define callbacks for training
early_stopping = EarlyStopping(patience=3, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('legal_text_generation_model.h5', save_best_only=True)

# Train the model using teacher forcing
batch_size = 64
epochs = 20

# Prepare target sequences for teacher forcing (shifted by one time step)
judgment_sequences_shifted = np.roll(judgment_sequences, shift=-1, axis=1)
judgment_train_shifted, judgment_val_shifted = judgment_sequences_shifted[:split_index], judgment_sequences_shifted[split_index:]

model.fit(input_train, judgment_train_shifted, batch_size=batch_size, epochs=epochs,
        validation_data=(input_val, judgment_val_shifted),
        callbacks=[early_stopping, model_checkpoint])

# Now, you can use the trained model for inference on new cases, as described in the previous responses.


In [ ]:
# Function to generate text based on input
def generate_text(input_text, model, tokenizer, max_sequence_length):
    # Tokenize and pad the input text
    input_sequence = tokenizer.texts_to_sequences([input_text])
    input_sequence = pad_sequences(input_sequence, maxlen=max_sequence_length, padding='post')
    
    # Generate the initial prediction
    predicted_sequence = model.predict(input_sequence)
    
    # Convert the predicted sequence into text
    predicted_indices = np.argmax(predicted_sequence, axis=-1)
    predicted_text = tokenizer.sequences_to_texts(predicted_indices)
    
    return predicted_text

# Assuming you have a DataFrame df with 'input_text' column
input_text = df['input_text'][0]  # Replace [0] with the index of the text you want to use as input
predicted_output = generate_text(input_text, model, tokenizer, max_sequence_length)
print("Input Text:")
print(input_text)
print("\nPredicted Output:")
print(predicted_output)

<font family="Arial, sans-serif" size="8px" color="#D2E0FB">MACHINE TRANSLATION</font>


In [ ]:
# exclude = set(string.punctuation) # Set of all special characters
# # Remove all the special characters
# lines['english']=lines['english'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
# lines['hindi']=lines['hindi'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

In [ ]:
lines = df

In [ ]:
lines['Judgment'][0]

In [ ]:
lines.head()

In [ ]:
# Add start and end tokens to target sequences
lines['Judgment'] = lines['Judgment'].apply(lambda x : 'START_ '+ x + ' _END')

In [ ]:
lines['Judgment'][0]

In [ ]:
data['THE FACTS'] = data['THE FACTS'].apply(lambda x: ' '.join(x) if isinstance(x, list) else x)
data['THE LAW'] = data['THE LAW'].apply(lambda x: ' '.join(x) if isinstance(x, list) else x)

# Combine input columns (e.g., 'PROCEDURE', 'THE FACTS', 'THE LAW') into a single text column
# data['input_text'] = data['PROCEDURE'] + ' ' + data['THE FACTS'] + ' ' + data['THE LAW']
data['input_text'] = data['THE FACTS'] + ' ' + data['THE LAW']


# Ensure all values in 'input_text' are strings
data['input_text'] = data['input_text'].astype(str)

In [ ]:
### Get English and Hindi Vocabulary
all_input_words=set()
for eng in lines['input_text']:
    for word in eng.split():
        if word not in all_input_words:
            all_input_words.add(word)

all_judgment_words=set()
for hin in lines['Judgement']:
    for word in hin.split():
        if word not in all_judgment_words:
            all_judgment_words.add(word)

In [ ]:
lines.head()

## continue from here

In [ ]:
lines['length_input']=lines['input_text'].apply(lambda x:len(x.split(" ")))
lines['length_judgment']=lines['Judgement'].apply(lambda x:len(x.split(" ")))

In [ ]:
lines.head()
lines[lines['length_input']>30].shape

In [ ]:
lines=lines[lines['length_input']<=20]
lines=lines[lines['length_judgment']<=20]

In [ ]:
print("maximum length of Hindi Sentence ",max(lines['length_judgment']))
print("maximum length of English Sentence ",max(lines['length_input']))

In [ ]:
max_length_src=max(lines['length_judgment'])
max_length_tar=max(lines['length_input'])

In [ ]:
input_words = sorted(list(all_input_words))
target_words = sorted(list(all_judgment_words))
num_encoder_tokens = len(all_input_words)
num_decoder_tokens = len(all_judgment_words)
num_encoder_tokens, num_decoder_tokens

In [ ]:
num_decoder_tokens

In [ ]:
num_decoder_tokens += 1

In [ ]:
num_decoder_tokens

In [ ]:
input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])

In [ ]:
input_token_index

In [ ]:
reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())

In [ ]:
reverse_input_char_index

In [ ]:
lines.head(10)

In [ ]:
from sklearn.model_selection import train_test_split
X, y = lines['input_text'], lines['Judgment']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,random_state=42)
X_train.shape, X_test.shape

In [ ]:
X_train

In [ ]:
encoder_input_data = np.zeros((2, max_length_src),dtype='float32')
decoder_input_data = np.zeros((2, max_length_tar),dtype='float32')
decoder_target_data = np.zeros((2, max_length_tar, num_decoder_tokens),dtype='float32')

In [ ]:
def generate_batch(X = X_train, y = y_train, batch_size = 128):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_token_index[word] # encoder input seq
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = target_token_index[word] # decoder input seq
                    if t>0:
                        # decoder target sequence (one hot encoded)
                        # does not include the START_ token
                        # Offset by one timestep
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

In [ ]:
latent_dim = 300
# Encoder
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(num_encoder_tokens+1, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [ ]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens+1, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
model.summary()
train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 64
epochs = 10

In [ ]:
model.save('eng-to-hindi.h5')

In [ ]:
a, b = next(generate_batch())

In [ ]:
b

In [ ]:
X_train[4]

In [ ]:
model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),
                    steps_per_epoch = train_samples/batch_size,
                    epochs=10,
                    validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
                    validation_steps = val_samples/batch_size)

In [ ]:
train_gen = generate_batch(X_train, y_train, batch_size = 1)
k=-1

In [ ]:
# Encode the input sequence to get the "thought vectors"
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2= dec_emb_layer(decoder_inputs) # Get the embeddings of the decoder sequence

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2) # A dense softmax layer to generate prob dist. over the target vocabulary

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

In [ ]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > 50):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

In [ ]:
from nltk.translate.bleu_score import sentence_bleu
ref = [
    'this is moonlight'.split(),
    'Look, this is moonlight'.split(),
    'moonlight it is'.split()
]
test = 'it is moonlight'.split()
print('BLEU score for test-> {}'.format(sentence_bleu(ref, test)))
 
test01 = 'it is cat and moonlight'.split()
print('BLEU score for test01-> {}'.format(sentence_bleu(ref, test01)))